In [1]:
import multiprocessing
import jieba
import tensorflow as tf
import tensorflow_hub as hub
import pandas as pd
import numpy as np
#import seaborn as sb
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
import pickle as pickle
%matplotlib inline

ImportError: No module named jieba

## data

In [9]:
tokenizer = (lambda string: " ".join([token for token in jieba.cut(string, cut_all=False)]))

In [3]:
path = "../Data/full/"
path_io = "./io/"

train_data = pd.read_csv(path + 'data_train.csv')
train_data.shape

(1710856, 8)

In [4]:
print train_data.fact.iloc[0]

公诉机关指控：2016年3月28日20时许，被告人颜某在本市洪山区马湖新村足球场马路边捡拾到被害人谢某的VIVOX5手机一部，并在同年3月28日21时起，分多次通过支付宝小额免密支付功能，秘密盗走被害人谢某支付宝内人民币3723元。案发后，被告人颜某家属已赔偿被害人全部损失，并取得谅解。公诉机关认为被告人颜某具有退赃、取得谅解、自愿认罪等处罚情节，建议判处被告人颜某一年以下××、××或者××，并处罚金。


In [17]:
min_count =170000
acc_wc = train_data.groupby('accusation').size()
selected_accusation = set((acc_wc[acc_wc >= min_count]).index)
train_data = train_data[train_data.accusation.isin(selected_accusation)]
%time train_data.fact = train_data.fact.apply(tokenizer)

CPU times: user 20min 52s, sys: 18.2 s, total: 21min 11s
Wall time: 21min 4s


In [20]:
%time print tokenizer(train_data.fact.iloc[0])

公诉   机关   指控   ：   2016   年   3   月   28   日   20   时许   ，   被告人   颜某   在   本市   洪山区   马湖   新村   足球场   马路边   捡拾   到   被害人   谢某   的   VIVOX5   手机   一部   ，   并   在   同年   3   月   28   日   21   时起   ，   分   多次   通过   支付宝   小额   免密   支付   功能   ，   秘密   盗走   被害人   谢某   支付宝   内   人民币   3723   元   。   案发后   ，   被告人   颜某   家属   已   赔偿   被害人   全部   损失   ，   并   取得   谅解   。   公诉   机关   认为   被告人   颜某   具有   退赃   、   取得   谅解   、   自愿   认罪   等   处罚   情节   ，   建议   判处   被告人   颜某   一年   以下   ×   ×   、   ×   ×   或者   ×   ×   ，   并   处罚金   。
CPU times: user 3.74 ms, sys: 1.8 ms, total: 5.54 ms
Wall time: 4.05 ms


min_count =100000

col = 'accusation'

df_tmp = train_data.groupby(col).filter(lambda x: len(x) > min_count).drop_duplicates(subset=col)
keep_list = df_tmp[col].tolist()
#keep_list = [x for x in keep_list if  ',' not in x]
train_data = train_data[train_data[col].isin(keep_list)]

col = 'relevant_articles'

df_tmp = train_data.groupby(col).filter(lambda x: len(x) > min_count).drop_duplicates(subset=col)
keep_list = df_tmp[col].tolist()
#keep_list = [x for x in keep_list if  ',' not in x]
train_data = train_data[train_data[col].isin(keep_list)]

train_data.shape

In [21]:
label_encoder = LabelEncoder()

train_data.accusation = label_encoder.fit_transform(train_data.accusation)
with open(path_io+'label_encoder_accu.pickle', 'wb') as handle:
    pickle.dump(label_encoder, handle, protocol=pickle.HIGHEST_PROTOCOL)
le_n_class_accu = label_encoder.classes_.shape[0]

# train_data.relevant_articles = label_encoder.fit_transform(train_data.relevant_articles)
# with open(path_io+'label_encoder_articles.pickle', 'wb') as handle:
#     pickle.dump(label_encoder, handle, protocol=pickle.HIGHEST_PROTOCOL)
le_n_class_articles = label_encoder.classes_.shape[0]

train_data.shape, le_n_class_accu, le_n_class_articles

((886581, 8), 3, 3)

In [29]:
train_data.to_csv("../Data/sample/data_tokenized_large_group.csv", index=False, encoding="utf-8")

## Models

train_data.accusation = train_data.accusation.map(int)
train_data.relevant_articles = train_data.relevant_articles.map(int)

droplist = ['accusation','criminals','punish_of_money','relevant_articles','death_penalty','imprisonment','life_imprisonment']
droplist_imprisonment = ['fact', 'accusation','criminals','relevant_articles','death_penalty','imprisonment','life_imprisonment']



train_input_fn_imprisonment = tf.estimator.inputs.pandas_input_fn(train_data.drop(droplist_imprisonment, axis =1), 
                                                              train_data['imprisonment'], shuffle=True, 
                                                              num_epochs=None, batch_size=512)

In [23]:
# tf.gfile.DeleteRecursively(path_io +'model_accu/')
# tf.gfile.DeleteRecursively(path_io +'model_articles/')
#tf.gfile.DeleteRecursively(path_io +'model_imprisonment/')


### accu model

In [24]:
label_encoder.classes_

array(['\xe5\x8d\xb1\xe9\x99\xa9\xe9\xa9\xbe\xe9\xa9\xb6',
       '\xe6\x95\x85\xe6\x84\x8f\xe4\xbc\xa4\xe5\xae\xb3',
       '\xe7\x9b\x97\xe7\xaa\x83'], dtype=object)

In [25]:
pd.value_counts(train_data.accusation)

2    363225
0    334103
1    189253
Name: accusation, dtype: int64

In [26]:
droplist = ['accusation','criminals','punish_of_money','relevant_articles','death_penalty','imprisonment','life_imprisonment']

train_input_fn_accu = tf.estimator.inputs.pandas_input_fn(train_data[['fact']], 
                                                          train_data['accusation'], shuffle=True, 
                                                          num_epochs=None, batch_size=256)

n_classes = le_n_class_accu
model_path = path_io+'model_accu/'
if tf.gfile.Exists(model_path):
    tf.gfile.DeleteRecursively(model_path)
feature_columns = [hub.text_embedding_column('fact','https://tfhub.dev/google/nnlm-zh-dim128-with-normalization/1', trainable=False)]
model_accu = tf.estimator.DNNClassifier([512, 512, 128], optimizer=tf.train.AdamOptimizer(learning_rate=1e-4),
                                   feature_columns=feature_columns, 
                                n_classes=n_classes, 
                                        model_dir=model_path)
model_accu.train(input_fn=train_input_fn_accu, max_steps=3000, )

INFO:tensorflow:Using /var/folders/3f/qq6s0gzj57qcmdhvndwjxhnmdtywv4/T/tfhub_modules to cache modules.
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_global_id_in_cluster': 0, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x12b6e9e10>, '_evaluation_master': '', '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_num_ps_replicas': 0, '_tf_random_seed': None, '_master': '', '_device_fn': None, '_num_worker_replicas': 1, '_task_id': 0, '_log_step_count_steps': 100, '_model_dir': './io/model_accu/', '_train_distribute': None, '_save_summary_steps': 100}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Initialize variable dnn/input_from_feature_columns/input_layer/fact_hub_module_embedding/module/embeddings/part_0:0 from checkpoint /var/folders/3f/qq6s0gzj57qcmdhvndwjxh

In [27]:
test_input_fn_accu = tf.estimator.inputs.pandas_input_fn(train_data[['fact']], 
                                                          train_data['accusation'], shuffle=False, 
                                                          num_epochs=1, batch_size=512)

model_accu.evaluate(input_fn=test_input_fn_accu )

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Initialize variable dnn/input_from_feature_columns/input_layer/fact_hub_module_embedding/module/embeddings/part_0:0 from checkpoint /var/folders/3f/qq6s0gzj57qcmdhvndwjxhnmdtywv4/T/tfhub_modules/8a7d79f6fd9af6ea61fa3e11a09a79a613503992/variables/variables with embeddings
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-07-12-09:46:50
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./io/model_accu/model.ckpt-3000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-07-12-09:47:51
INFO:tensorflow:Saving dict for global step 3000: accuracy = 0.6210465, average_loss = 1.4837204, global_step = 3000, loss = 759.49097
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 3000: ./io/model_accu/model.ckpt-3000


{'accuracy': 0.6210465,
 'average_loss': 1.4837204,
 'global_step': 3000,
 'loss': 759.49097}

### articles model

In [24]:
droplist = ['accusation','criminals','punish_of_money','relevant_articles','death_penalty','imprisonment','life_imprisonment']

train_input_fn_articles = tf.estimator.inputs.pandas_input_fn(train_data.drop(droplist, axis =1), 
                                                              train_data['relevant_articles'], shuffle=True, 
                                                              num_epochs=None, batch_size=512)

n_classes = le_n_class_articles
model_path = path_io+'model_articles/'
feature_columns = [hub.text_embedding_column('fact','https://tfhub.dev/google/nnlm-zh-dim128-with-normalization/1', trainable=True)]
model_articles = tf.estimator.DNNClassifier([512, 512, 128], 
                                   feature_columns=feature_columns, n_classes=n_classes, model_dir=model_dir)
model_articles.train(input_fn=train_input_fn_articles, max_steps=100)

NameError: name 'model_dir' is not defined

### imprisonment model

In [52]:
droplist = ['fact', 'accusation','criminals','relevant_articles','death_penalty','imprisonment','life_imprisonment']



train_input_fn_imprisonment = tf.estimator.inputs.pandas_input_fn(train_data.drop(droplist, axis =1), 
                                                              train_data['imprisonment'], shuffle=True, 
                                                              num_epochs=None, batch_size=512)


model_path = path_io+'model_imprisonment/'
punish_of_money = tf.feature_column.numeric_column("punish_of_money")
feature_columns = [punish_of_money]
model_imprisonment = tf.estimator.LinearRegressor(model_dir=model_path, feature_columns=feature_columns)
model_imprisonment.train(input_fn=train_input_fn_imprisonment, max_steps=1000)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/home/aida-zw/Desktop/ws/notebook/io/model_imprisonment/', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f39c89ef198>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into /home/aida-zw/Des

## Prediction Class

In [15]:
import tensorflow as tf
import tensorflow_hub as hub
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import pickle as pickle

class Predictor(object):
    def __init__(self):
        
        #path_io = "predictor/"
        path_io = "/home/aida-zw/Desktop/ws/notebook/io/"
        
        df_accu_dir = path_io + "accu.csv"
        df_articles_dir = path_io + "law.csv"

        self.df_accu = pd.read_csv(df_accu_dir)
        self.df_articles = pd.read_csv(df_articles_dir)

        self.n_classes_accu = 190
        self.n_classes_articles = 242

        self.model_dir_accu = path_io + 'model_accu/'
        self.model_dir_articles = path_io + 'model_articles/'

        label_encoder_dir_accu = path_io + 'label_encoder_accu.pickle'
        label_encoder_dir_articles = path_io + 'label_encoder_articles.pickle'

        hub_dir = path_io + 'google_nnlm-zh-dim128-with-normalization_1'


        with open(label_encoder_dir_accu, 'rb') as handle:
            self.label_encoder_accu = pickle.load(handle)


        with open(label_encoder_dir_articles, 'rb') as handle:
            self.label_encoder_articles = pickle.load(handle)
        
        self.feature_columns = [hub.text_embedding_column('fact',hub_dir , trainable=True)]

        self.model_accu = tf.estimator.DNNClassifier([512, 512, 128],
                                                     feature_columns=self.feature_columns, 
                                                     n_classes=self.n_classes_accu, 
                                                     model_dir=self.model_dir_accu)
        
        self.model_articles = tf.estimator.DNNClassifier([512, 512, 128],
                                                     feature_columns=self.feature_columns, 
                                                     n_classes=self.n_classes_articles, 
                                                     model_dir=self.model_dir_articles)
    
    def predict(self, content):

        test_data = pd.DataFrame({'fact': content})
        ans_accusation, ans_accusation_arr = self.predict_accu(test_data)
        ans_articles, ans_articles_arr = self.predict_articles(test_data)
        ans_imprisonment = self.predict_time(test_data)
#         ans = pd.DataFrame({'accusation':ans_accusation, 'articles': ans_articles, 'imprisonment':ans_imprisonment,
#                            'accusation_arr':ans_accusation_arr, 'articles_arr': ans_articles_arr})
        ans = pd.DataFrame({'accusation':ans_accusation, 'articles': ans_articles, 'imprisonment':ans_imprisonment})
        dict_ans = ans.to_dict(orient='records')
        return dict_ans
        
        
    def predict_time(self, test_data):
        time = []
        for i in range(test_data.shape[0]):
            time.append(7)
        return time
    
    
    def predict_articles(self, test_data):
        test_input_fn = tf.estimator.inputs.pandas_input_fn(
            test_data, 
            None, 
            shuffle=False, 
            batch_size=128)
        result = list(self.model_articles.predict(input_fn=test_input_fn))
        result_class = np.array([p['classes'][0] for p in result])
        prediction_arr = self.label_encoder_articles.inverse_transform(result_class.astype(int))
        articles_num = self.get_articles_num(self.df_articles, prediction_arr)
        return articles_num, prediction_arr
    
    def predict_accu(self, test_data):
        test_input_fn = tf.estimator.inputs.pandas_input_fn(
            test_data, 
            None, 
            shuffle=False, 
            batch_size=128)
        result = list(self.model_accu.predict(input_fn=test_input_fn))
        result_class = np.array([p['classes'][0] for p in result])
        prediction_arr = self.label_encoder_accu.inverse_transform(result_class.astype(int))
        accusation_num = self.get_accusation_num(self.df_accu, prediction_arr)
        
        return accusation_num, prediction_arr
    
    def get_accusation_num(self, df_accu, arr):
        keys = df_accu.accusation.tolist()
        keys = [x.strip() for x in keys]
        values = df_accu.accusation_num.tolist()
        dict_accu = dict(zip(keys, values))

        accusation_num = list()
        for accu in arr:
            accusation_num.append([dict_accu.get(x.replace("'", "").strip()) for x in accu.split(',')])
        return accusation_num
    
    def get_articles_num(self, df_articles, arr):
        keys = df_articles.articles.tolist()
        values = df_articles.articles_num.tolist()
        dict_articles = dict(zip(keys, values))

        articles_num = list()
        for art in arr:
            articles_num.append([dict_articles.get(int(x)) for x in art.split(',')])
        return articles_num


## Test Prediction Class
### if __name__ == "__main__":

In [20]:
path_sample = "/home/aida-zw/Desktop/ws/Data/sample/"
test_data = pd.read_csv(path_sample + "data_test.csv")

content = test_data['fact'].tolist()    
    
model = Predictor()

result = model.predict(content)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/home/aida-zw/Desktop/ws/notebook/io/model_accu/', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f3d722ce9e8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/home/aida-zw/Desktop/ws/notebook/io/model_articles/', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, 

/home/aida-zw/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Initialize variable dnn/input_from_feature_columns/input_layer/fact_hub_module_embedding/module/embeddings/part_0:0 from checkpoint b'/home/aida-zw/Desktop/ws/notebook/io/google_nnlm-zh-dim128-with-normalization_1/variables/variables' with embeddings
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /home/aida-zw/Desktop/ws/notebook/io/model_articles/model.ckpt-100
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


/home/aida-zw/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [21]:
result

[{'accusation': [191], 'articles': [58], 'imprisonment': 7},
 {'accusation': [191], 'articles': [58], 'imprisonment': 7},
 {'accusation': [191], 'articles': [58], 'imprisonment': 7},
 {'accusation': [190], 'articles': [58], 'imprisonment': 7},
 {'accusation': [191], 'articles': [58], 'imprisonment': 7},
 {'accusation': [190], 'articles': [58], 'imprisonment': 7},
 {'accusation': [191], 'articles': [58], 'imprisonment': 7},
 {'accusation': [190], 'articles': [58], 'imprisonment': 7},
 {'accusation': [191], 'articles': [58], 'imprisonment': 7},
 {'accusation': [190], 'articles': [58], 'imprisonment': 7},
 {'accusation': [190], 'articles': [58], 'imprisonment': 7},
 {'accusation': [190], 'articles': [58], 'imprisonment': 7},
 {'accusation': [191], 'articles': [58], 'imprisonment': 7},
 {'accusation': [190], 'articles': [58], 'imprisonment': 7},
 {'accusation': [190], 'articles': [58], 'imprisonment': 7},
 {'accusation': [191], 'articles': [58], 'imprisonment': 7},
 {'accusation': [190], '

In [ ]:
def predict(self, content):
    result = []
    for a in range(0, len(content)):
        result.append({
            "accusation": [1, 2, 3],
            "imprisonment": 5,
            "articles": [5, 7, 9]
        })
    return result